In [1]:
import csv
import os
import json
import pandas as pd
import plotly.express as px
import numpy as np

In [2]:
from app.flipside.models import Flipside
from app.constants import oracle_feeds_lookup, oracle_feeds, phase_windows, filenames_filtered, timestamp_windows

In [3]:
from app.visuals.plotly_helper import combined_action_layout, combined_action_layout_time

In [4]:

def fig_append_phase_labels(fig, start = None, end=None):
    for window in phase_windows:
        if window[0]>= start and window[1]<= end and window[3] != "":
            fig.add_vrect(x0=window[0], x1=window[1], 
                        annotation_text=window[2], annotation_position="top left",
                        fillcolor=window[3], opacity=0.25, line_width=0)
    return fig

In [ ]:
fs = Flipside(filenames_filtered)

In [6]:
def process_layout(target_address, window_start, window_end, title, tokens=True):
    deposits = fs.dataframes['aave_deposits']
    withdraws = fs.dataframes['aave_withdraws']
    borrows = fs.dataframes['aave_borrows']
    repayments = fs.dataframes['aave_repayments']
    liquidations = fs.dataframes['aave_liquidations']

    deposits = deposits[deposits['depositor_address']== target_address]
    withdraws = withdraws[withdraws['depositor_address']== target_address]
    borrows = borrows[borrows['borrower_address']== target_address]
    repayments = repayments[repayments['borrower']== target_address]
    liquidations = liquidations[liquidations['borrower']== target_address]

    deposits = fs.apply_window_df(deposits, window_start, window_end)
    withdraws = fs.apply_window_df(withdraws, window_start, window_end)
    borrows = fs.apply_window_df(borrows, window_start, window_end)
    repayments = fs.apply_window_df(repayments, window_start, window_end)
    liquidations = fs.apply_window_df(liquidations, window_start, window_end)

    combined_action_layout(title, 
                        deposits, withdraws, 
                        borrows, repayments, 
                        liquidations,
                        tokens,
                        # phase_windows, 
                        # window_start, window_end
                        )



In [7]:
def process_layout_time(target_address, window_start, window_end, title, tokens=True):
    deposits = fs.dataframes['aave_deposits']
    withdraws = fs.dataframes['aave_withdraws']
    borrows = fs.dataframes['aave_borrows']
    repayments = fs.dataframes['aave_repayments']
    liquidations = fs.dataframes['aave_liquidations']

    deposits = deposits[deposits['depositor_address']== target_address]
    withdraws = withdraws[withdraws['depositor_address']== target_address]
    borrows = borrows[borrows['borrower_address']== target_address]
    repayments = repayments[repayments['borrower']== target_address]
    liquidations = liquidations[liquidations['borrower']== target_address]


    deposits = fs.apply_window_df(deposits, window_start, window_end)
    withdraws = fs.apply_window_df(withdraws, window_start, window_end)
    borrows = fs.apply_window_df(borrows, window_start, window_end)
    repayments = fs.apply_window_df(repayments, window_start, window_end)
    liquidations = fs.apply_window_df(liquidations, window_start, window_end)

    combined_action_layout_time(title, 
                        deposits, withdraws, 
                        borrows, repayments, 
                        liquidations,
                        tokens,
                        # phase_windows, 
                        # window_start, window_end
                        )


# Quick View

## AVI

In [8]:
target_address = '0x57e04786e231af3343562c062e0d058f25dace9e'
window_start = 15962578 - 7*(7200)
window_end = 16027236 + 10

title = "Avi"

In [ ]:
process_layout(target_address, window_start, window_end, title)
# fig = fig_append_phase_labels(fig, window_start, window_end)
# fig.show()

In [10]:
process_layout_time(target_address, window_start, window_end, title+" USD", False)

In [ ]:
process_layout(target_address, window_start, window_end, title)

## Michael

In [12]:
target_address = '0x7a16ff8270133f063aab6c9977183d9e72835428'
# window_start = 15962578 - 420
# window_end = 15962578 + 420

title = "Michael"

In [ ]:
process_layout(target_address, window_start, window_end, title)

In [14]:
process_layout_time(target_address, window_start, window_end, title+" USD", False)

In [ ]:
process_layout(target_address, window_start, window_end, title)

In [ ]:
((16025650 - 16025280) * 12) /60

In [17]:
# window_start = 15962578 - 14*(7200)
# window_end = 16027236 + 2*(7200)

crv_address = oracle_feeds['CRV'].lower()
dai_address = oracle_feeds['DAI'].lower()

oracles = fs.apply_window('oracles', window_start, window_end)
oracle_set = oracles[oracles["feed_address"].isin([dai_address, crv_address])]


In [55]:
def process_oracle_adj(
        target, 
        target_oracle_address, 
        dai_oracle_address = '0x773616E4d11A78F511299002da57A0a94577F1f4'.lower()
    ):
    block_records = {
        'block_number': [], 
        f"{target}_price": [], 
        'dai_price': [], 
        f"{target}_priced_in_dai": [], 
        'block_timestamp': []
        }
    for index, row in oracle_set.iterrows():
        block_number = row['block_number'] 
        block_timestamp = row['block_timestamp']
        if not row['latest_answer_unadj']:
            dai = None
            crv = None
        elif row['feed_address'] == target_oracle_address:
            crv = float(row['latest_answer_unadj']) / 10**18
            dai = None
        elif row['feed_address'] == dai_oracle_address:
            dai = float(row['latest_answer_unadj']) / 10**18
            crv = None
        else: 
            dai = None
            crv = None
        # print (block_records)
        # print(block_number)
        if crv:
            block_records['crv_price'].append(crv)
        elif dai:
            block_records['dai_price'].append(dai)

        if crv or dai:
            if len(block_records['block_number']) == 0:
                block_records['block_number'].append(block_number)
                block_records['block_timestamp'].append(block_timestamp)
            elif not block_number == block_records['block_number'][-1]:
                block_records['block_number'].append(block_number)
                block_records['block_timestamp'].append(block_timestamp)


            if len(block_records['crv_price']) == len(block_records['dai_price']) and len(block_records['crv_price']) > 0:
                last_crv_price = block_records['crv_price'][-1]
                last_dai_price = block_records['dai_price'][-1]
                block_records['crv_priced_in_dai'].append(last_crv_price/last_dai_price)
    return block_records

In [ ]:
block_records = process_oracle_adj('crv', crv_address, dai_address)
print(block_records.keys())

for key in block_records.keys():
    print(len(block_records[key]))

In [ ]:
price_feed = pd.DataFrame(block_records)

price_feed.keys()

In [62]:
target_address = '0x57e04786e231af3343562c062e0d058f25dace9e'
# window_start = 15962578 - 14*(7200)
# window_end = 16027236 + 2*(7200)

title = "Avi"

# target_address = '0x7a16ff8270133f063aab6c9977183d9e72835428'
window_start = 16023400 - 420
window_end = 17837124 + 420

title = "Michael"

target_address = '0x7a16ff8270133f063aab6c9977183d9e72835428'

In [ ]:
deposits = fs.dataframes['aave_deposits']
withdraws = fs.dataframes['aave_withdraws']
borrows = fs.dataframes['aave_borrows']
repayments = fs.dataframes['aave_repayments']
liquidations = fs.dataframes['aave_liquidations']


deposits = deposits[deposits['depositor_address']== target_address]
withdraws = withdraws[withdraws['depositor_address']== target_address]
borrows = borrows[borrows['borrower_address']== target_address]
repayments = repayments[repayments['borrower']== target_address]
liquidations = liquidations[liquidations['borrower']== target_address]


deposits = fs.apply_window_df(deposits, window_start, window_end)
withdraws = fs.apply_window_df(withdraws, window_start, window_end)
borrows = fs.apply_window_df(borrows, window_start, window_end)
repayments = fs.apply_window_df(repayments, window_start, window_end)
liquidations = fs.apply_window_df(liquidations, window_start, window_end)
deposits


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


fig = make_subplots(specs=[[{"secondary_y": True}]])
# fig = go.Figure()
fig.update_layout(
    title=f"Aave Combined Actions: {title} (USD)",
#     xaxis_title="X Axis Title",
#     yaxis_title="Y Axis Title",
#     legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)


fig = fig.add_trace(
    go.Scatter(
        x = deposits.block_timestamp,
        y = deposits.supplied_usd, 
        name = "Deposited Tokens",
        mode='markers',
        marker_color="green",
    ),
    secondary_y=False
)


fig = fig.add_trace(
    go.Scatter(
        x = withdraws.block_timestamp,
        y = withdraws.withdrawn_usd, 
        name = "Withdrawn Tokens",
        mode='markers',
        marker_color="blue"
    ),
    secondary_y=False
)



fig = fig.add_trace(
    go.Scatter(
        x = borrows.block_timestamp,
        y = borrows.borrowed_usd, 
        name = "Borrowed Tokens",
        mode='markers',
        marker_color="orange",
    ),
    secondary_y=False
)

fig = fig.add_trace(
    go.Scatter(
        x=repayments.block_timestamp, 
        y=repayments.repayed_usd, 
        name="Repayed Tokens",
        mode='markers',
        marker_color='black',
    ),
    secondary_y=False
)

fig = fig.add_trace(
    go.Scatter(
        x=liquidations.block_timestamp,
        y=liquidations.liquidated_amount_usd,
        name="Liquidated Collateral",
        mode='markers',
        marker_color='red',
    ),
    secondary_y=False
)

fig = fig.add_trace(
    go.Scatter(
        x=liquidations.block_timestamp,
        y=liquidations.debt_to_cover_amount_usd,
        name="Liquidated Debt Covered",
        mode='markers',
        marker_color='purple',
    ),
    secondary_y=False
)
# fig = fig.add_trace(
#     go.Scatter(
#         x=price_feed.block_timestamp,
#         y=price_feed.crv_priced_in_dai,
#         name="Curve Price",
#     ),
#     secondary_y=True

# )

# fig.add_vrect(x0=timestamp_windows['avi_initial_deposit'], x1=timestamp_windows['avi_attack_start'], 
#             annotation_text='Avi Setup', annotation_position="top left",
#             fillcolor='orange', opacity=0.25, line_width=0)

fig.add_vrect(x0=timestamp_windows['avi_attack_start'], x1=timestamp_windows['avi_initial_liquidation'], 
            annotation_text='Attack', annotation_position="top left",
            fillcolor='blue', opacity=0.25, line_width=0)

fig.add_vrect(x0=timestamp_windows['avi_initial_liquidation'], x1=timestamp_windows['avi_last_liquidation'], 
            annotation_text='Liq', annotation_position="top left",
            fillcolor='purple', opacity=0.25, line_width=0)


# fig.add_vrect(x0=timestamp_windows['michael_ramp_up'], x1=timestamp_windows['michael_ramp_stop'], 
#             annotation_text='M Setup', annotation_position="top left",
#             fillcolor='blue', opacity=0.25, line_width=0)
fig.show()